<a href="https://colab.research.google.com/github/Francrodi/AprendAut_T2/blob/master/Cliente_Lab2_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# Descarga las stopwords si aún no las tienes
nltk.download('stopwords')
nltk.download('punkt')

# delete date and time
METADATA = r'^\d+\/\d+\/\d+, \d+:\d+ - '
AUTHOR = r'^[^:]+:\s*'
LAUGH = r'(?:[aA]*(?:[jJ][aA]*)+[jJ]?)\b'

def preprocess(path: str):
    # Lee el archivo de conversaciones de WhatsApp
    with open(path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Elimina la fecha y hora de cada mensaje
    text = re.sub(METADATA, '', text, flags=re.MULTILINE)

    # Initialize an empty string to store the processed text
    lines = text.splitlines()
    text = ""
    for line in lines:
        # ignore lines with <Media omitted>
        if "<Media omitted>" in line or "<Multimedia omitido>" in line:
            continue
        # delete author and :
        elif ":" in line:
            line = re.sub(AUTHOR, '', line)
            text += line #+ "\n"

    # Elimina 3 letras repetidas, ej: 'hholaaa' -> 'hhola'
    # no elimine dos por el tema de palabras con doble r, doble l, etc.
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    # Elimina risas
    text = re.sub(LAUGH, '', text)

    # Tokenización: Divide el texto en palabras o términos
    tokens = text.split(" ")

    # Conversión a minúsculas y eliminación de signos de puntuación
    clean_tokens = [token.lower() for token in tokens]

    # Eliminación de stopwords (palabras comunes que generalmente no aportan información)
    filtered_tokens = [token for token in clean_tokens if token not in set(stopwords.words('spanish'))]

    # Ahora, 'filtered_tokens' contiene las palabras procesadas listas para ser utilizadas en tu modelo.
    return filtered_tokens

raw_vocabulary = preprocess('chat.txt')

df = pd.DataFrame(raw_vocabulary, columns=['word'])
# exclude words of 1 letter
counts_df = df[df['word'].str.len() > 1]
# make a new df ['word', 'occurrences']
vocabulary = counts_df.groupby('word').size().reset_index(name='occurrences')

mapeo = {
    'toy': 'estoy',
    'tamos': 'estamos',
    'tamo': 'estamos',
    'pal': 'para',
    'vamo': 'vamos',
    'tan': 'estan',
    'tas': 'estas',
}
to_exclude = [ 'pa', 'bo', 'ta', ]

voc2_df = vocabulary.copy()
# replace words
voc2_df['word'] = voc2_df['word'].replace(mapeo)
# exclude words
voc2_df = voc2_df[~voc2_df['word'].isin(to_exclude)]

voc2_df.sort_values(by='occurrences', ascending=False).head(n=10)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,word,occurrences
4041,si,181
1694,error,66
2247,hacer,60
1577,eliminó,60
487,alguien,57
2167,grupo,54
1256,da,45
4028,ser,42
4124,solo,41
3293,parte,39


In [2]:
import re

URL = r"https?://\S+"
MENTION = r"@\S+"

filename = "chat.txt"

with open(filename, "r", encoding="utf-8") as file:
  data = file.readlines()

messages_list = []

for line in data:
  text = line
  if "-" in line:
    text = text.split("-", maxsplit=1)[1]
    if ":" in text:
      text = text.split(":", maxsplit=1)[1]
  message = text.strip()
  messages_list.append(message)

def preprocess_chat(chat: list) -> list:
  chat = [message.lower() for message in chat]
  chat = [re.sub(URL, "", message) for message in chat]
  chat = [re.sub(r"[^a-zñáéíóú\s]", "", message) for message in chat]
  chat = [re.sub(URL, "", message) for message in chat]
  chat = [re.sub(MENTION, "", message) for message in chat]
  chat = [message.strip() for message in chat]
  chat = [message for message in chat if message != ""]
  chat = [re.sub(r"\s+", " ", message) for message in chat]
  chat = list(dict.fromkeys(chat))
  chat.sort()
  return chat

messages_list = preprocess_chat(messages_list)

messages_list

['a',
 'a alguien la falta grupo',
 'a alguno le falta grupo',
 'a cuántas cifras significativas',
 'a esto',
 'a hacerme taylor de segundo orden',
 'a hs',
 'a l',
 'a la izquierda elegí la carpeta donde queres hacer los m',
 'a la tuya es mejor',
 'a las notas',
 'a mi',
 'a mi al principio no me salia nada pero ahora entiendo bastante mas',
 'a mi la a dice ambos menores que epsilon de maquina',
 'a mi le dio algox la resta entre lo de la gráfica y lo de la aprox del error',
 'a mi me dio a la nada que ver',
 'a mi me dio eso si',
 'a mi me dio que c se encuentra en el intervalo siendo el error absoluto',
 'a mi me dió igual despejé h y al compararlo con el h de la gráfica de la parte b me dió con sentido',
 'a mi me facilitó un montón me tocó un buen docente pero entregar los ejercicios y ver que onda me ayudó a darme cuenta que soluciones buscan ellos y te entrena muchísimo para el parcial',
 'a mi me gusta',
 'a mi me paso lo mismo en la primera',
 'a mi me quedo ese n',
 'a mi m

In [3]:
# Crear una lista con todas las palabras de los mensajes
words_list = []

for message in messages_list:
  words = message.split(" ")
  for word in words:
    words_list.append(word)

# Crear un diccionario con las palabras y la cantidad de veces que aparecen
words_dict = {}

for word in words_list:
  if word in words_dict:
    words_dict[word] += 1
  else:
    words_dict[word] = 1

words_dict

{'a': 426,
 'alguien': 87,
 'la': 514,
 'falta': 25,
 'grupo': 107,
 'alguno': 31,
 'le': 80,
 'cuántas': 1,
 'cifras': 4,
 'significativas': 3,
 'esto': 18,
 'hacerme': 1,
 'taylor': 17,
 'de': 676,
 'segundo': 4,
 'orden': 12,
 'hs': 2,
 'l': 6,
 'izquierda': 2,
 'elegí': 4,
 'carpeta': 7,
 'donde': 17,
 'queres': 6,
 'hacer': 67,
 'los': 154,
 'm': 10,
 'tuya': 1,
 'es': 344,
 'mejor': 10,
 'las': 179,
 'notas': 47,
 'mi': 58,
 'al': 104,
 'principio': 3,
 'no': 469,
 'me': 216,
 'salia': 1,
 'nada': 35,
 'pero': 172,
 'ahora': 35,
 'entiendo': 15,
 'bastante': 10,
 'mas': 27,
 'dice': 41,
 'ambos': 8,
 'menores': 3,
 'que': 737,
 'epsilon': 2,
 'maquina': 2,
 'dio': 19,
 'algox': 1,
 'resta': 18,
 'entre': 15,
 'lo': 270,
 'gráfica': 9,
 'y': 400,
 'aprox': 2,
 'del': 155,
 'error': 79,
 'ver': 27,
 'eso': 114,
 'si': 318,
 'c': 34,
 'se': 191,
 'encuentra': 2,
 'en': 433,
 'el': 687,
 'intervalo': 6,
 'siendo': 4,
 'absoluto': 19,
 'dió': 11,
 'igual': 68,
 'despejé': 1,
 'h': 30,

In [4]:
filename = "es.txt"

with open(filename, "r", encoding="utf-8") as file:
  data = file.readlines()

# Crear un diccionario con las palabras en español
spanish_dict = {}

for line in data:
  word = line.strip()
  spanish_dict[word] = True

spanish_dict

{'aaronica': True,
 'aaronicas': True,
 'aaronico': True,
 'aaronicos': True,
 'aaronita': True,
 'aaronitas': True,
 'aba': True,
 'ababa': True,
 'ababas': True,
 'ababilla': True,
 'ababillaba': True,
 'ababillabais': True,
 'ababillabamos': True,
 'ababillaban': True,
 'ababillabas': True,
 'ababillado': True,
 'ababillais': True,
 'ababillamos': True,
 'ababillan': True,
 'ababillando': True,
 'ababillar': True,
 'ababillara': True,
 'ababillarais': True,
 'ababillaramos': True,
 'ababillaran': True,
 'ababillaras': True,
 'ababillare': True,
 'ababillareis': True,
 'ababillaremos': True,
 'ababillaren': True,
 'ababillares': True,
 'ababillaria': True,
 'ababillariais': True,
 'ababillariamos': True,
 'ababillarian': True,
 'ababillarias': True,
 'ababillaron': True,
 'ababillarse': True,
 'ababillas': True,
 'ababillase': True,
 'ababillaseis': True,
 'ababillasemos': True,
 'ababillasen': True,
 'ababillases': True,
 'ababillaste': True,
 'ababillasteis': True,
 'ababille': Tru

In [5]:
# Lista de palabras unicas (Vocabulario)
vocabulary = list(dict.fromkeys(words_list))

# Cruzamos vocabulario con diccionario de palabras en español
vocabulary = [word for word in vocabulary if word in spanish_dict]

vocabulary

['alguien',
 'la',
 'falta',
 'grupo',
 'alguno',
 'le',
 'cifras',
 'significativas',
 'esto',
 'de',
 'segundo',
 'orden',
 'izquierda',
 'carpeta',
 'donde',
 'queres',
 'hacer',
 'los',
 'tuya',
 'es',
 'mejor',
 'las',
 'notas',
 'mi',
 'al',
 'principio',
 'no',
 'me',
 'salia',
 'nada',
 'pero',
 'ahora',
 'entiendo',
 'bastante',
 'mas',
 'dice',
 'ambos',
 'menores',
 'que',
 'epsilon',
 'maquina',
 'dio',
 'resta',
 'entre',
 'lo',
 'del',
 'error',
 'ver',
 'eso',
 'si',
 'se',
 'encuentra',
 'en',
 'el',
 'intervalo',
 'siendo',
 'absoluto',
 'igual',
 'con',
 'parte',
 'sentido',
 'un',
 'buen',
 'docente',
 'entregar',
 'ejercicios',
 'onda',
 'cuenta',
 'soluciones',
 'buscan',
 'ellos',
 'te',
 'entrena',
 'para',
 'parcial',
 'gusta',
 'paso',
 'mismo',
 'primera',
 'quedo',
 'ese',
 'final',
 'estaba',
 'bien',
 'sin',
 'mucho',
 'mayor',
 'analizar',
 'tiene',
 'como',
 'hiciste',
 'yo',
 'use',
 'formula',
 'teorico',
 'dando',
 'por',
 'vi',
 'calcularon',
 'usando

In [6]:
def build_pdh_dict(chat: list, N=1):
    pdh_dict = {}
    for message in chat:
        words = message.split(" ")
        if len(words) > N: # Si el mensaje tiene menos de N palabras no se toma en cuenta TODO ver si tiene sentido o usar > 1
            for key, val in enumerate(words):
                if (val not in pdh_dict.keys()): pdh_dict[val] = {}
                if (key >= N):
                    N_words_before = words[key-N:key]
                    for word_before in N_words_before:
                        pdh_dict[val][word_before] = 1 if not word_before in pdh_dict[val] else pdh_dict[val][word_before] + 1
    for key in pdh_dict.keys():
        for subkey in pdh_dict[key].keys():
            pdh_dict[key][subkey] /= words_dict[subkey]
    return pdh_dict

pdh_dict = build_pdh_dict(messages_list, 1)
pdh_dict

{'a': {'la': 0.011673151750972763,
  'dio': 0.15789473684210525,
  'ayudó': 1.0,
  'dió': 0.09090909090909091,
  'clase': 0.025,
  'es': 0.011627906976744186,
  'marketing': 0.5,
  'va': 0.5581395348837209,
  'utiliza': 0.5,
  'entonces': 0.022222222222222223,
  'refería': 1.0,
  'voy': 0.9375,
  'ahora': 0.05714285714285714,
  'vayan': 0.5,
  'gracias': 0.07272727272727272,
  'van': 0.75,
  'cuanto': 0.16666666666666666,
  'que': 0.009497964721845319,
  'mañana': 0.22727272727272727,
  'pusiste': 0.5,
  'grupo': 0.009345794392523364,
  'recomiendan': 0.3333333333333333,
  'subirlo': 1.0,
  'uno': 0.043478260869565216,
  'ayuda': 0.3333333333333333,
  'si': 0.050314465408805034,
  'buenas': 0.06756756756756757,
  'parte': 0.3170731707317073,
  'h': 0.03333333333333333,
  'sabe': 0.047619047619047616,
  'agregar': 1.0,
  'hoy': 0.14285714285714285,
  'consultarles': 1.0,
  'presencial': 0.5,
  'buenass': 0.2857142857142857,
  'cursando': 0.3333333333333333,
  'sí': 0.047619047619047616,

In [7]:
# Construimos diccionario de probabilidades de las palabras P(H)
total_words = sum(words_dict.values())
ph_dict = {word: frequency / total_words for word, frequency in words_dict.items()}

In [8]:
# TODO: Ver prob cuando no existe palabra en entrenamiento
def recomendacion_bayesiana(frase):
  N = 1
  if len(frase) >= N:
    keys = frase[-N:]
  else:
    keys = frase

  keys = [k.lower() for k in keys]

  # Bucar la palabra con mayor probabilidad, es decir, que maximice P(H|D) = P(D|H) * P(H) (Hipoteses MAP)
  max_prob = 0
  palabra_sugerida = ""
  for word in vocabulary:
    prob = None
    for key in keys:
      prob = pdh_dict.get(word, {}).get(key, 0.001) if prob is None else prob * pdh_dict.get(word, {}).get(key, 0.001)
    prob *= ph_dict[word]
    if prob >= max_prob:
      max_prob = prob
      palabra_sugerida = word
  #TODO Si la palabra anterior es una stopword NO recomendar otra stopoword
  return palabra_sugerida

def preprocesar_chat(chat):
  # algo aca
  pass

def entrenar():
  # Tener en cuenta el N
  # Armar una estructura tipo diccionario de profundidad N
  pass



##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      print("----- Comenzando frase nueva -----")
      frase = []

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra)

    if frase:
      palabra_sugerida = recomendacion_bayesiana(frase)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))



# hola chau buen dia PRED

# p(buen|PRED)*p(dia|PRED)
# p[PRED]["dia"]*p[PRED]["buen"]*p(PRED)



Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.
>> profesores
profesores dijeron
>> 
profesores dijeron que
>> 
profesores dijeron que no
>> 
profesores dijeron que no es
>> 
profesores dijeron que no es el
>> 
profesores dijeron que no es el de
>> 
profesores dijeron que no es el de la
>> 
profesores dijeron que no es el de la de
>> 
profesores dijeron que no es el de la de la
>> 
profesores dijeron que no es el de la de la de
>> 
profesores dijeron que no es el de la de la de la
>> 
profesores dijeron que no es el de la de la de la de


KeyboardInterrupt: ignored